## MC simulations of fluids

:::{admonition} **What we will learn**

- **Monte Carlo sampling**: Random sampling of configurations with correct Boltzmann probability.
- **Minimum image convention**: Essential for periodic boundary conditions.
- **Statistical ensemble is implemented via Metropolis conditions**: Simulating canonical ensemble (constant $NVT$) by accepting or rejecting moves via comparions of Botlzman weights
- **Structural characterization**: Compute $g(r)$ to understand spatial correlations.

:::

## LJ model of a simple fluids, noble gases


**Check out reference values of [LJ fluid properties tabulated by NIST](https://www.nist.gov/mml/csd/chemical-informatics-research-group/lennard-jones-fluid-properties)**




### Monte Carlo Simulation of a Lennard-Jones Fluid


- This simulation models a **Lennard-Jones fluid** using the **Metropolis Monte Carlo (MC)** algorithm in **canonical ensemble** (constant $NVT$).

- Particles are placed in a **periodic cubic box** and random moves are proposed and accepted or rejected based on the **Boltzmann probability**.

- This simulation is organized into a **clean object-oriented structure** where:
  - The system state (positions, box size, temperature) is handled inside a **`LJ_MC_System`** class.
  - The simulation tracks **total energy** and **pair correlation function** $ g(r) $ over time.


### Lennard-Jones (LJ) Potential

Each pair of particles interacts through the LJ potential:

$$
V(r) = 4 \left( \frac{1}{r^{12}} - \frac{1}{r^6} \right)
$$

where:
- $r$ is the distance between two particles.
- The potential captures **short-range repulsion** and **long-range attraction** typical of simple fluids.

A **cutoff distance** $r_c$ is used to make calculations more efficient.



### Metropolis Monte Carlo Algorithm

At each step:
1. Randomly pick a particle.
2. Propose a random displacement.
3. Compute the change in energy $\Delta E$.
4. Accept or reject the move based on the Metropolis criterion:

$$
P_{\text{accept}} = \min\left(1, e^{-\beta \Delta E}\right)
$$


- If the move is rejected, the particle returns to its previous position.

### [Periodic Boundary conditions and minimum image criterion](https://en.wikipedia.org/wiki/Periodic_boundary_conditions)

- If a particle leaves the box, it re-enters at the opposite side.
- What should be the distance between partiles? There is some mbiguity because distance between atoms may now include crossing the system boundary and re-entering. This is identical to introducing copies of the particles around the simulation box. We adopt **minimum image convention** by choosing the shortest distance possible

![](./pbc_fig1.gif)


:::{figure-md} 

<img src="./figs/pbc1.png" class="bg-primary" width="400px">  

Perioidc Boundary Conditions
::: 



:::{figure-md} 

<img src="./figs/pbc2.png" class="bg-primary" width="400px">  

Minimum Image Convention showing that we only track particles with clsoest distance. 
::: 




In [ ]:
def pbc_basic(x, L):
  '''Wrap a single components into [-L/2, L/2) assuming box centered at origin'''
    
  if   x >=  L/2: x -= L  

  elif x <= -L/2: x += L

  return x


@njit
def pbc_wrap(r_vec, L):
    """
    Wrap all vector components into [-L/2, L/2) assuming box centered at origin.
    """
    
    return (r_vec + L/2) % L - L/2

### Lennar Jones potential

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

@widgets.interact(sig=(1,2, 0.1),eps=(0.1,3))
def plot_lj(sig=1, eps=1):
    
    r  = np.linspace(0.5, 3, 1000)
    lj = 4 * eps *  ( (sig/r)**12 -(sig/r)**6 )
    
    plt.plot(r, lj, lw=3)
    plt.plot(r, -4*eps*(sig/r)**6, '-',lw=3)
    plt.plot(r,  4*eps*(sig/r)**12,'-',lw=3)

    plt.ylim([-3,3])
    plt.xlabel(r'$r$')
    plt.ylabel(r'$U(r)$')
    plt.legend(['LJ', 'Attr-LJ','Repuls-LJ' ])
    plt.show()

### Code Structure

The simulation is organized into a **single class**: `LJ_MC_System`.

Each function of the class has a clear role:


#### 1. `__init__(self, rho=0.88, N_cell=3, T=1.0, rcut=2.5)`

**Purpose**: Initialize the system.

- Set up the **system parameters**:
  - Number of particles $N$, box length $L$, temperature $T$.
- Arrange particles initially in a **face-centered cubic (FCC)** lattice to avoid overlaps.
- Initialize **pair indices** for efficient distance calculations.
- Store parameters like **cutoff distance** $r_c$ and **inverse temperature** $\beta$.


#### 2. `minimum_image(rij, L)`

**Purpose**:  
Apply the **minimum image convention** to correctly compute distances across periodic boundaries.

Formula:

$$
\mathbf{r}_{ij} = \mathbf{r}_{ij} - L \times \text{round}(\mathbf{r}_{ij}/L)
$$



#### 3. `particle_energy(idx)`

**Purpose**:  

Compute the **interaction energy** between a single particle and all others.

- Needed to evaluate the energy change $\Delta E$ during a move.
- Excludes self-interactions.
- Only includes pairs within cutoff radius $r_c$.


#### 4. `total_energy()`

**Purpose**:  
Compute the **total potential energy** of the entire system.

- Loops over all unique particle pairs.
- Applies the Lennard-Jones potential with cutoff.


#### 5. `mc_move(max_disp=0.1)`

**Purpose**:  

Perform a **single Metropolis MC move**:

- Randomly choose a particle.
- Propose a random displacement up to `max_disp`.
- Accept or reject the move based on the energy difference and Metropolis criterion.


#### 6. `sample()`

**Purpose**:  
Compute a **histogram** of pair distances.

- Used for calculating the **pair correlation function** $g(r)$.
- Histograms the distance between all unique particle pairs.


#### 7. `simulate(nsteps=50000, freq_out=500)`

**Purpose**:  
Run the full **Monte Carlo simulation loop**.

- Repeat `mc_move()` for `nsteps` steps.
- Every `freq_out` steps:
  - Record the total energy.
  - Sample the pair distance histogram.
- Return arrays of recorded energies and histograms.

### MC engine for LJ fluid in 3D (NVT ensemble)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

class LJ_MC_System:
    def __init__(self, rho=0.88, N_cell=3, T=1.0, rcut=2.5):
        # System parameters
        self.N = 4 * N_cell**3
        self.rho = rho
        self.L = (self.N / rho)**(1/3)
        self.T = T
        self.beta = 1.0 / T
        self.rcut = rcut
        self.rcut_sq = rcut**2

        # Create FCC lattice positions
        base = np.array([[0, 0, 0], [0.5, 0.5, 0], [0.5, 0, 0.5], [0, 0.5, 0.5]])
        pos = np.array([b + [i, j, k] for i in range(N_cell) for j in range(N_cell) for k in range(N_cell) for b in base])
        self.pos = pos * (self.L / N_cell)

        # Pair indices
        self.I, self.J = np.triu_indices(self.N, k=1)

    def minimum_image(self, r_vec):
        return (r_vec + self.L/2) % self.L - self.L/2

    def total_energy(self):
        r_vec = self.minimum_image(self.pos[self.I] - self.pos[self.J])
        r_sq = np.sum(r_vec**2, axis=1)
        mask = r_sq < self.rcut_sq
        r2_inv = 1.0 / r_sq[mask]
        r6_inv = r2_inv**3
        return 4 * np.sum(r6_inv * (r6_inv - 1))

    def particle_energy(self, idx):
        rij = self.minimum_image(self.pos[idx] - self.pos)
        r_sq = np.sum(rij**2, axis=1)
        mask = (r_sq < self.rcut_sq) & (r_sq > 0)  # exclude self-interaction
        r2_inv = 1.0 / r_sq[mask]
        r6_inv = r2_inv**3
        return 4 * np.sum(r6_inv * (r6_inv - 1))

    def mc_move(self, max_disp=0.1):
        idx = np.random.randint(self.N)
        old_pos = self.pos[idx].copy()
        old_energy = self.particle_energy(idx)

        # Propose move
        displacement = (np.random.rand(3) - 0.5) * 2 * max_disp
        self.pos[idx] = (self.pos[idx] + displacement) % self.L

        new_energy = self.particle_energy(idx)
        dE = new_energy - old_energy

        # Metropolis criterion
        if np.random.rand() > np.exp(-self.beta * dE):
            # Reject move
            self.pos[idx] = old_pos

    def sample(self):
        """ Sample pair distances for g(r) """
        r_vec = self.minimum_image(self.pos[self.I] - self.pos[self.J])
        r_sq = np.sum(r_vec**2, axis=1)
        hist, _ = np.histogram(np.sqrt(r_sq), bins=30, range=(0, self.L/2))
        return hist

    def simulate(self, nsteps=50000, freq_out=500):
        hists = []
        energies = []

        for step in range(nsteps):
            self.mc_move()

            if step % freq_out == 0:
                hist = self.sample()
                hists.append(hist)
                energies.append(self.total_energy())

        return np.array(energies), np.mean(hists, axis=0)

### Running MCMC simulation on LJ system

In [ ]:
lj_mc = LJ_MC_System(rho=0.88, N_cell=3, T=1.0)

# Run simulation
energies, hist = lj_mc.simulate(nsteps=100000, freq_out=500)

# Post-processing
r = np.linspace(0, lj_mc.L/2, 30)
dr = r[1] - r[0]
shell_volumes = 4*np.pi*r**2*dr
g_r = hist / (lj_mc.N * lj_mc.rho * shell_volumes)

# Plot
plt.plot(r, g_r, '-o')
plt.xlabel(r'$r$')
plt.ylabel(r'$g(r)$')
plt.show()


### Problems

**1. Vary sim parameters**

- **Tune maximum displacement** (`max_disp`) to optimize acceptance rate (~40%-50% is ideal).
- **Check how $g(r)$** changes when density or temperature changes.
- **Implement energy cutoffs with shifted potentials** to smooth out the energy at $r_c$.
- **Try simulating hard spheres** (infinite repulsion at contact).
 
**2. Run MC simulations of LJ fluid at several temperatures to identify critical temperature.**

 - At each temperature evaluate heat capacity and RDFs.
 - Plot how energy, heat capacity and RDF change as a function of temperature.
 - Study dependence on sampling efficiency on magnitude of particle displacement.